In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

Data folder is /media/luca/DATA/DATASETS
Experiment save directory is  /media/luca/DATA/EXPERIMENTS


In [3]:
far.utils._check()

4


In [4]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])

v2 = tf.Variable([[-1., -2], [1., 0.]])

lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape(), scalar=False)

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

io_optim = far.AdamOptimizer(far.get_hyperparameter('lr', 0.01), epsilon=1.e-6) # same epsilon of rfho
# io_optim = far.GradientDescentOptimizer(0.1)

# io_optim_dict = io_optim.minimize(cost) 

oo = tf.reduce_mean(v1*v2)
optim_oo = tf.train.AdamOptimizer()

In [5]:
farho = far.HyperOptimizer() 

In [6]:
run = farho.minimize(oo, optim_oo, cost, io_optim)

In [7]:
# tf.global_variables()

In [8]:
# far.utils.hyperparameters()

In [9]:
tf.global_variables_initializer().run()
ss.run(far.utils.hyperparameters())

[array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.0099999998]

In [10]:
optim_dict = list(farho.hypergradient._optimizer_dicts)[0]

In [11]:
# optim_dict.dynamics

### check the dynamics

In [12]:
for _ in range(5):
#     print(ss.run(io_optim._get_beta_accumulators()))
#     print(ss.run(io_optim._lr_t))
    _res_dy = ss.run(optim_dict.dynamics)
    _res_1 = ss.run(optim_dict.iteration)
#     print(_res_1)
#     print(_res_dy)
    
#     print()
    print([np.linalg.norm(a - b) for a, b in zip(_res_dy, _res_1)])
#     print()
    print()
    
    

[2.9802322e-07, 0.0, 0.0, 2.0647654e-07, 0.0, 0.0, 0.0, 0.0]

[2.3841858e-07, 0.0, 2.3283064e-10, 2.0647654e-07, 0.0, 0.0, 0.0, 0.0]

[1.7881393e-07, 0.0, 0.0, 2.0647654e-07, 8.4293696e-08, 3.7252903e-09, 0.0, 0.0]

[1.1920929e-07, 0.0, 0.0, 1.4600097e-07, 0.0, 7.4505806e-09, 0.0, 0.0]

[1.1920929e-07, 0.0, 3.7252903e-09, 1.4600097e-07, 0.0, 0.0, 0.0, 0.0]



The difference is due to the diferent positioning of the $\varepsilon$  (inside the square root)

In [13]:
run(10, _skip_hyper_ts=True)

In [14]:
weights_optim_far = ss.run([v1, v2])

In [15]:
ss.run(far.utils.hyperparameters())

[array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.0099999998]

In [16]:
rev_hypergrads = ss.run(far.utils.hypergradients())
rev_hypergrads

[array([[  1.74622983e-09,  -1.86264515e-09],
        [ -1.74622983e-09,   0.00000000e+00]], dtype=float32), 11.965221]

In [17]:
ss.run(lmbd)

array([[ 1.,  1.],
       [ 1.,  1.]], dtype=float32)

# check with rfho

In [18]:
import rfho as rf

Environment variable RFHO_EXP_FOLDER not found. Current directory will be used
Experiment save directory is  /media/luca/DATA/Progs/FAR-HO/tests/Experiments
Environment variable RFHO_DATA_FOLDER not found. Variables HELP_WIN and HELP_UBUNTU contain info.
Data folder is /media/luca/DATA/Progs/FAR-HO/tests


In [19]:
w, c, co = rf.vectorize_model([v1, v2], cost, oo, augment=2)

In [20]:
lr = tf.Variable(0.01, name='lr')
dyn = rf.AdamOptimizer.create(w, lr, loss=c)

In [21]:
# w.var_list()[0].assign(1.)

In [22]:
hyperg = rf.HyperOptimizer(dyn, {co: [lmbd, lr]}, rf.ReverseHG)

In [23]:
hyperg.initialize()

True

In [24]:
hgs = hyperg.run(10, val_feed_dict_suppliers={co:lambda: None},
                 _debug_no_hyper_update=True)

In [25]:
print('rfho')
print(ss.run(hyperg.hyper_gradients.hyper_gradient_vars))
print('far')
print(ss.run(far.utils.hypergradients()))

rfho
[array([[  1.68111001e-06,   1.34526636e-06],
       [ -1.68111001e-06,   0.00000000e+00]], dtype=float32), 11.965094]
far
[array([[  1.74622983e-09,  -1.86264515e-09],
       [ -1.74622983e-09,   0.00000000e+00]], dtype=float32), 11.965221]


Difference are due to the different implementation of the two methods. far.AdamOptimizer uses the correct dynamics for updating the weights and the ''incorrect'' one, i.e. with $\sqrt{v - \varepsilon}$ instead of $\sqrt{v} + \varepsilon$,   for computing the hypergradients while rfho.Adam uses the ''incorrect'' also for the updates.

In [26]:
print(ss.run(w.var_list()[0]))

[ 0.90036476  2.90011191 -0.90035355 -1.90016878  0.90035355  0.        ]


In [27]:
weights_optim_far

[array([ 0.90035141,  2.9001112 ], dtype=float32),
 array([[-0.90035063, -1.90016854],
        [ 0.90035063,  0.        ]], dtype=float32)]

### CHECK WITH FORWARD

In [28]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])

v2 = tf.Variable([[-1., -2], [1., 0.]])

lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape(), scalar=True)

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

io_optim = far.AdamOptimizer(far.get_hyperparameter('lr', 0.01), epsilon=1.e-6) # same epsilon of rfho
# io_optim = far.GradientDescentOptimizer(0.1)

# io_optim_dict = io_optim.minimize(cost) 

oo = tf.reduce_mean(v1*v2)
optim_oo = tf.train.AdamOptimizer()


In [29]:
farho_fw = far.HyperOptimizer(far.ForwardHG())
run_fw = farho_fw.minimize(oo, optim_oo, cost, io_optim)

In [30]:
tf.global_variables_initializer().run()
run_fw(10, _skip_hyper_ts=True)

In [31]:
ss.run(far.utils.hypergradients())

[1.362591e-09, -1.0128521e-09, -1.362591e-09, 0.0, 11.965221]

In [32]:
rev_hypergrads

[array([[  1.74622983e-09,  -1.86264515e-09],
        [ -1.74622983e-09,   0.00000000e+00]], dtype=float32), 11.965221]

Some very small differences... but acceptable.. (derivative w.r.t. learning rate is the same!) 